In [1]:
import sys
print(sys.version)
print(spark.version)

3.8.5 | packaged by conda-forge | (default, Aug 29 2020, 01:22:49) 
[GCC 7.5.0]
3.0.1


In [2]:
import subprocess

from pyspark.sql.functions import *
from pyspark.sql.types import *

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [3]:
spark.conf.set("spark.sql.repl.eagerEval.enabled",True)

In [4]:
%%time

influence_df = spark.read.parquet('gs://file-name')

CPU times: user 3.78 ms, sys: 787 µs, total: 4.57 ms
Wall time: 6.62 s


In [5]:
influence_df.count()

13672943

In [8]:
influence_score = influence_df.select([influence_df.user_id, 
                                       influence_df.retweeted_status,
                                       influence_df.retweet_count,
                                       influence_df.quote_count,
                                       influence_df.favorite_count])

In [9]:
#create retweet count + quote count column and total engagement column
influence_score = influence_score\
    .withColumn("retweet_quote_count", col("retweet_count")+col("quote_count"))\
    .withColumn("total_engagement", col("retweet_count")+col("quote_count")+col("favorite_count"))

In [10]:
influence_score

user_id,retweeted_status,retweet_count,quote_count,favorite_count,retweet_quote_count,total_engagement
56952466,"[,, Wed Nov 03 03...",17,2,90,19,109
1377329837191405578,null,0,0,0,0,0
290039056,"[,, Tue Nov 02 22...",1312,39,3851,1351,5202
3646453759,"[,, Wed Nov 03 04...",1,0,0,1,1
53567660,null,0,0,0,0,0
309570860,"[,, Tue Nov 02 21...",40,0,91,40,131
1921974067,"[,, Tue Nov 02 19...",15950,1387,84938,17337,102275
1170715225756188672,"[,, Tue Nov 02 21...",129,20,272,149,421
2875819794,"[,, Tue Nov 02 14...",1218,59,5356,1277,6633
862342470360866816,null,0,0,0,0,0


In [11]:
#create total tweet counts by user
total_counts = influence_score.groupBy('user_id').count().alias('total_count')

In [12]:
total_counts

user_id,count
3305808923,38
40362526,2
404412177,45
1314113540,5
1667792210,1
1336451673053679620,44
200688599,328
1436036421437968391,1
3332152301,2
24198047,50


In [13]:
#join both dataframes
joined_df = influence_score.join(total_counts, "user_id" ,"left")

In [14]:
joined_df2 = joined_df.withColumn("influence", (col("retweet_quote_count")/col("total_engagement")) * col("count"))

In [15]:
#fill in nulls in influence columns
joined_df3 = joined_df2.na.fill(value=0,subset=["influence"])

In [16]:
joined_df4 = joined_df3.withColumn("influence", when(joined_df3.retweeted_status.isNull(), joined_df3.influence*3).otherwise(joined_df3.influence))

In [18]:
#joined_df4

In [19]:
influencers = joined_df4.select(['user_id','influence']).groupBy("user_id").agg(avg("influence").alias("avg_influence"))

In [21]:
#influencers

In [37]:
influencers.count()

2729351

In [88]:
influencers.write.format("parquet")\
.mode('overwrite')\
.save('gs://msca-bdp-students-bucket/shared_data/abharathsingh/influencers')

print('done')

done


In [38]:
joined_df5 = influencers.join(joined_df4, 
                       "user_id",
                       "left")

In [39]:
joined_df5.count()

13672943

In [74]:
#drop variables that we don't need
joined_df6 = joined_df5.drop("retweet_count")\
    .drop("quote_count")\
    .drop("favorite_count")\
    .drop("influence")\
    .drop("retweeted_status")\
    .drop("retweet_quote_count")\
    .drop("total_engagement")

In [75]:
joined_df6.printSchema()

root
 |-- user_id: long (nullable = true)
 |-- avg_influence: double (nullable = true)
 |-- count: long (nullable = true)



In [76]:
joined_df7 = joined_df6.dropDuplicates(['user_id'])

In [77]:
joined_df7

user_id,avg_influence,count
964,0.0,2
38543,0.0,1
48763,0.4011765885616873,1
60033,0.0,3
747203,0.0,3
781154,0.0,3
806000,0.0,1
814261,1.6957981959907849,5
843931,0.4111409653584109,1
1070071,0.0,1


In [78]:
joined_df7.count()

2729351

In [87]:
from pyspark.sql import DataFrameStatFunctions as statFunc
statFunc(joined_df7).approxQuantile("avg_influence", [0.5], 0)

Py4JJavaError: An error occurred while calling o428.approxQuantile.
: org.apache.spark.sql.catalyst.errors.package$TreeNodeException: execute, tree:
Exchange hashpartitioning(user_id#3L, 200), true, [id=#10062]
+- Project [user_id#3L, retweeted_status#8, (retweet_count#9L + quote_count#7L) AS retweet_quote_count#49L, ((retweet_count#9L + quote_count#7L) + favorite_count#10L) AS total_engagement#56L]
   +- FileScan parquet [user_id#3L,quote_count#7L,retweeted_status#8,retweet_count#9L,favorite_count#10L] Batched: false, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex[gs://msca-bdp-students-bucket/shared_data/abharathsingh/covid_tweets], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<user_id:bigint,quote_count:bigint,retweeted_status:struct<contributors:string,coordinates:...

	at org.apache.spark.sql.catalyst.errors.package$.attachTree(package.scala:56)
	at org.apache.spark.sql.execution.exchange.ShuffleExchangeExec.doExecute(ShuffleExchangeExec.scala:151)
	at

In [79]:
#join onto dataframe to have all new columns
covid_influence = influence_df.join(joined_df7, 
                       "user_id",
                       "left")

In [80]:
covid_influence.schema.names

['user_id',
 'created_at',
 'id',
 'user',
 'user_name',
 'user_description',
 'followers_count',
 'quote_count',
 'retweeted_status',
 'retweet_count',
 'favorite_count',
 'text',
 'verified_user',
 'avg_influence',
 'count']

In [82]:
#covid_influence

In [83]:
covid_influence.count()

13672943

In [84]:
covid_influence.write.format("parquet")\
.mode('overwrite')\
.save('gs://folder-name')
print('done')

done
